In [2]:
# ===============================
# PART 1 – DESCRIPTIVE STATISTICS
# 1) Load and inspect
# ===============================

# Imports (kept minimal for this section)
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  # used later for plots

# ---------- CONFIG ----------
INPUT_CSV = "smaller_online_retail_clean.csv"
EXCLUDE_ID_COLS = {"CustomerID", "InvoiceNo", "StockCode"}  # identifiers (not analyzed/plotted)

# ---------- LOAD ----------
print(f"Working directory: {os.getcwd()}")
if not os.path.exists(INPUT_CSV):
    raise FileNotFoundError(
        f"Could not find {INPUT_CSV}. Place it in this working directory or update INPUT_CSV."
    )

df = pd.read_csv(INPUT_CSV)
print(f"Loaded file: {INPUT_CSV}")
print(f"Shape (rows, cols): {df.shape}\n")

# ---------- QUICK PEEK ----------
print("First 10 rows:")
display(df.head(10))

print("\nColumn dtypes:")
display(df.dtypes)

# ---------- EXCLUDE IDENTIFIERS ----------
# Treat CustomerID, InvoiceNo, and StockCode as identifiers only (no stats/plots).
present_ids = [c for c in EXCLUDE_ID_COLS if c in df.columns]
if present_ids:
    print(f"\nIdentifier columns detected and excluded from analysis: {present_ids}")
else:
    print("\nNo identifier columns from the exclusion list were found.")

df_analysis = df.drop(columns=present_ids, errors="ignore")

# ---------- WHAT'S NUMERIC AFTER EXCLUSION ----------
numeric_cols = df_analysis.select_dtypes(include="number").columns.tolist()
print("\nNumeric columns available for descriptive statistics (after excluding IDs):")
print(numeric_cols)

# Helpful expectations for this dataset:
expected_num = {"Quantity", "UnitPrice", "SalesAmount"}
missing_expected = sorted(list(expected_num.difference(set(numeric_cols))))
if missing_expected:
    print("\n[Note] These expected numeric columns were not found as numeric "
          f"(or are missing): {missing_expected}")


Working directory: /Users/suhasrahul/MEGA/MIS/SEM_3/BI/A5
Loaded file: smaller_online_retail_clean.csv
Shape (rows, cols): (5398, 11)

First 10 rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,SalesAmount,InvoiceMonth,IsReturn
0,580816,21378,SMALL TALL CAMPHOR WOOD TOADSTOOL,12,2011-12-06 11:28:00,0.39,15157.0,United Kingdom,4.68,2011-12,False
1,556129,23190,BUNDLE OF 3 SCHOOL EXERCISE BOOKS,48,2011-06-09 10:46:00,1.45,14060.0,United Kingdom,69.60,2011-06,False
2,561614,85099B,JUMBO BAG RED RETROSPOT,10,2011-07-28 12:50:00,2.08,13871.0,United Kingdom,20.80,2011-07,False
3,577358,23241,TREASURE TIN GYMKHANA DESIGN,3,2011-11-18 15:59:00,4.13,NaN,United Kingdom,12.39,2011-11,False
4,561515,15058B,PINK POLKADOT GARDEN PARASOL,1,2011-07-27 15:16:00,10.79,NaN,United Kingdom,10.79,2011-07,False
5,559506,22358,KINGS CHOICE TEA CADDY,1,2011-07-08 15:17:00,2.46,NaN,United Kingdom,2.46,2011-07,False
6,570646,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,6,2011-10-11 12:49:00,7.95,12824.0,United Kingdom,47.70,2011-10,False
7,563038,84508A,CAMOUFLAGE DESIGN TEDDY,240,2011-08-11 15:05:00,2.10,16656.0,United Kingdom,504.00,2011-08,False
8,580677,22887,NUMBER TILE VINTAGE FONT 8,1,2011-12-05 14:40:00,1.95,16200.0,United Kingdom,1.95,2011-12,False
9,C574026,51014C,"FEATHER PEN,COAL BLACK",-1,2011-11-02 12:26:00,0.39,14606.0,United Kingdom,-0.39,2011-11,True



Column dtypes:


InvoiceNo        object
StockCode        object
Description      object
Quantity          int64
InvoiceDate      object
UnitPrice       float64
CustomerID      float64
Country          object
SalesAmount     float64
InvoiceMonth     object
IsReturn           bool
dtype: object


Identifier columns detected and excluded from analysis: ['CustomerID', 'StockCode', 'InvoiceNo']

Numeric columns available for descriptive statistics (after excluding IDs):
['Quantity', 'UnitPrice', 'SalesAmount']


In [3]:
# ===============================
# PART 1 – DESCRIPTIVE STATISTICS
# step 2. Overall statistics table
# ===============================

# ---------- COMPUTE STATISTICS ----------
def compute_mad_from_mean(series):
    """
    Compute Mean Absolute Deviation (MAD) from the mean.
    MAD = mean(|x - mean(x)|)
    """
    series_clean = series.dropna()
    if len(series_clean) == 0:
        return np.nan
    mean_val = series_clean.mean()
    return (series_clean - mean_val).abs().mean()


# Build statistics for each numeric column
stats_list = []

for col in numeric_cols:
    series = df_analysis[col]
    
    # Basic counts and central tendency
    count_val = series.count()
    mean_val = series.mean()
    median_val = series.median()
    
    # Range statistics
    min_val = series.min()
    max_val = series.max()
    range_val = max_val - min_val
    
    # Variance and standard deviation (sample, ddof=1)
    var_val = series.var(ddof=1)
    std_val = series.std(ddof=1)
    
    # MAD from mean
    mad_val = compute_mad_from_mean(series)
    
    # Quartiles and IQR
    q1_val = series.quantile(0.25)
    q3_val = series.quantile(0.75)
    iqr_val = q3_val - q1_val
    
    # Shape statistics
    skew_val = series.skew()
    kurt_val = series.kurt()  # excess kurtosis
    
    # Collect into dictionary
    stats_dict = {
        'Column': col,
        'count': count_val,
        'mean': mean_val,
        'median': median_val,
        'min': min_val,
        'max': max_val,
        'range': range_val,
        'variance': var_val,
        'std_dev': std_val,
        'MAD_from_mean': mad_val,
        'Q1': q1_val,
        'Q3': q3_val,
        'IQR': iqr_val,
        'skewness': skew_val,
        'kurtosis': kurt_val
    }
    
    stats_list.append(stats_dict)

# ---------- CREATE DATAFRAME ----------
stats_df = pd.DataFrame(stats_list)

print("\n" + "="*80)
print("OVERALL DESCRIPTIVE STATISTICS TABLE")
print("="*80)
display(stats_df)

# ---------- SAVE TO FILE ----------
os.makedirs("ba_outputs", exist_ok=True)
output_path = "ba_outputs/descriptive_stats_report.csv"
stats_df.to_csv(output_path, index=False)
print(f"\n✓ Descriptive statistics saved to: {output_path}")

# ---------- OPTIONAL: PRINT FORMATTED TABLE ----------
print("\n" + "="*80)
print("FORMATTED STATISTICS SUMMARY")
print("="*80)
for col in numeric_cols:
    row = stats_df[stats_df['Column'] == col].iloc[0]
    print(f"\n{col}:")
    print(f"  Count:          {row['count']:.0f}")
    print(f"  Mean:           {row['mean']:.4f}")
    print(f"  Median:         {row['median']:.4f}")
    print(f"  Min:            {row['min']:.4f}")
    print(f"  Max:            {row['max']:.4f}")
    print(f"  Range:          {row['range']:.4f}")
    print(f"  Variance:       {row['variance']:.4f}")
    print(f"  Std Dev:        {row['std_dev']:.4f}")
    print(f"  MAD (mean):     {row['MAD_from_mean']:.4f}")
    print(f"  Q1:             {row['Q1']:.4f}")
    print(f"  Q3:             {row['Q3']:.4f}")
    print(f"  IQR:            {row['IQR']:.4f}")
    print(f"  Skewness:       {row['skewness']:.4f}")
    print(f"  Kurtosis:       {row['kurtosis']:.4f}")

print("\n" + "="*80)


OVERALL DESCRIPTIVE STATISTICS TABLE


,Column,count,mean,median,min,max,range,variance,std_dev,MAD_from_mean,Q1,Q3,IQR,skewness,kurtosis
0,Quantity,5398,10.060578,3.00,-144.00,1900.00,2044.00,1408.831053,37.534398,10.875939,1.00,11.00,10.00,28.134552,1248.044792
1,UnitPrice,5398,4.684524,2.10,0.06,2275.54,2275.48,1995.934744,44.675885,4.409201,1.25,4.13,2.88,46.387189,2291.411262
2,SalesAmount,5398,18.433662,9.75,-2185.04,3285.00,5470.04,5939.033287,77.065124,18.869027,3.38,17.70,14.32,17.171905,895.521642



✓ Descriptive statistics saved to: ba_outputs/descriptive_stats_report.csv

FORMATTED STATISTICS SUMMARY

Quantity:
  Count:          5398
  Mean:           10.0606
  Median:         3.0000
  Min:            -144.0000
  Max:            1900.0000
  Range:          2044.0000
  Variance:       1408.8311
  Std Dev:        37.5344
  MAD (mean):     10.8759
  Q1:             1.0000
  Q3:             11.0000
  IQR:            10.0000
  Skewness:       28.1346
  Kurtosis:       1248.0448

UnitPrice:
  Count:          5398
  Mean:           4.6845
  Median:         2.1000
  Min:            0.0600
  Max:            2275.5400
  Range:          2275.4800
  Variance:       1995.9347
  Std Dev:        44.6759
  MAD (mean):     4.4092
  Q1:             1.2500
  Q3:             4.1300
  IQR:            2.8800
  Skewness:       46.3872
  Kurtosis:       2291.4113

SalesAmount:
  Count:          5398
  Mean:           18.4337
  Median:         9.7500
  Min:            -2185.0400
  Max:            3285.

In [4]:
# ===============================
# PART 1 – DESCRIPTIVE STATISTICS
# step 3. Grouped statistics (choose one grouping)
# ===============================

# ---------- CHOOSE YOUR GROUPING ----------
# Uncomment exactly ONE of the following lines:

GROUPING_COLUMN = "InvoiceMonth"  # Seasonality analysis
# GROUPING_COLUMN = "Country"        # Regional differences
# GROUPING_COLUMN = "IsReturn"       # Returns vs. non-returns

print("\n" + "="*80)
print(f"GROUPED STATISTICS BY: {GROUPING_COLUMN}")
print("="*80)

# ---------- VALIDATE GROUPING COLUMN EXISTS ----------
if GROUPING_COLUMN not in df.columns:
    raise ValueError(
        f"Grouping column '{GROUPING_COLUMN}' not found in dataset. "
        f"Available columns: {df.columns.tolist()}"
    )

# ---------- COMPUTE GROUPED STATISTICS ----------
def compute_group_stats(dataframe, group_col, numeric_columns):
    """
    Compute grouped statistics for numeric columns by a grouping column.
    
    Returns a DataFrame with columns:
    - Group column
    - For each numeric column: count, mean, median, std, Q1, Q3, IQR
    """
    results = []
    
    # Group the data
    grouped = dataframe.groupby(group_col)
    
    for group_name, group_df in grouped:
        row_dict = {group_col: group_name}
        
        for num_col in numeric_columns:
            series = group_df[num_col]
            
            # Compute statistics
            count_val = series.count()
            mean_val = series.mean()
            median_val = series.median()
            std_val = series.std(ddof=1)
            q1_val = series.quantile(0.25)
            q3_val = series.quantile(0.75)
            iqr_val = q3_val - q1_val
            
            # Add to row with clear column naming
            row_dict[f"{num_col}_count"] = count_val
            row_dict[f"{num_col}_mean"] = mean_val
            row_dict[f"{num_col}_median"] = median_val
            row_dict[f"{num_col}_std"] = std_val
            row_dict[f"{num_col}_Q1"] = q1_val
            row_dict[f"{num_col}_Q3"] = q3_val
            row_dict[f"{num_col}_IQR"] = iqr_val
        
        results.append(row_dict)
    
    return pd.DataFrame(results)


# ---------- GENERATE GROUPED STATISTICS ----------
grouped_stats_df = compute_group_stats(df, GROUPING_COLUMN, numeric_cols)

print(f"\nNumber of groups found: {len(grouped_stats_df)}")
print(f"\nGroups: {grouped_stats_df[GROUPING_COLUMN].tolist()}\n")

# Display the table
display(grouped_stats_df)

# ---------- SAVE TO FILE ----------
os.makedirs("ba_outputs", exist_ok=True)
output_filename = f"grouped_stats_by_{GROUPING_COLUMN}.csv"
output_path = os.path.join("ba_outputs", output_filename)
grouped_stats_df.to_csv(output_path, index=False)
print(f"\n✓ Grouped statistics saved to: {output_path}")

# ---------- OPTIONAL: FORMATTED SUMMARY BY GROUP ----------
print("\n" + "="*80)
print("FORMATTED GROUPED STATISTICS SUMMARY")
print("="*80)

for _, row in grouped_stats_df.iterrows():
    group_val = row[GROUPING_COLUMN]
    print(f"\n{'='*80}")
    print(f"GROUP: {GROUPING_COLUMN} = {group_val}")
    print('='*80)
    
    for num_col in numeric_cols:
        print(f"\n  {num_col}:")
        print(f"    Count:    {row[f'{num_col}_count']:.0f}")
        print(f"    Mean:     {row[f'{num_col}_mean']:.4f}")
        print(f"    Median:   {row[f'{num_col}_median']:.4f}")
        print(f"    Std Dev:  {row[f'{num_col}_std']:.4f}")
        print(f"    Q1:       {row[f'{num_col}_Q1']:.4f}")
        print(f"    Q3:       {row[f'{num_col}_Q3']:.4f}")
        print(f"    IQR:      {row[f'{num_col}_IQR']:.4f}")

print("\n" + "="*80)

# ---------- QUICK COMPARISON ACROSS GROUPS ----------
print("\n" + "="*80)
print("QUICK COMPARISON: MEAN VALUES ACROSS GROUPS")
print("="*80)

comparison_df = grouped_stats_df[[GROUPING_COLUMN] + 
                                  [f"{col}_mean" for col in numeric_cols]].copy()
comparison_df.columns = [GROUPING_COLUMN] + [f"{col}_mean" for col in numeric_cols]
display(comparison_df)

print("\n" + "="*80)
print("QUICK COMPARISON: MEDIAN VALUES ACROSS GROUPS")
print("="*80)

comparison_df_median = grouped_stats_df[[GROUPING_COLUMN] + 
                                         [f"{col}_median" for col in numeric_cols]].copy()
comparison_df_median.columns = [GROUPING_COLUMN] + [f"{col}_median" for col in numeric_cols]
display(comparison_df_median)

print("\n" + "="*80)


GROUPED STATISTICS BY: InvoiceMonth

Number of groups found: 13

Groups: ['2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12']



,InvoiceMonth,Quantity_count,Quantity_mean,Quantity_median,Quantity_std,Quantity_Q1,Quantity_Q3,Quantity_IQR,UnitPrice_count,UnitPrice_mean,...,UnitPrice_Q1,UnitPrice_Q3,UnitPrice_IQR,SalesAmount_count,SalesAmount_mean,SalesAmount_median,SalesAmount_std,SalesAmount_Q1,SalesAmount_Q3,SalesAmount_IQR
0,2010-12,412,6.065534,2.0,13.754648,1.0,6.0,5.0,412,4.584660,...,1.45,4.24,2.79,412,14.083592,7.530,31.888232,3.320,17.0000,13.6800
1,2011-01,358,9.170391,3.0,19.112127,1.0,8.0,7.0,358,4.839162,...,1.25,4.13,2.88,358,17.116313,8.445,34.693547,3.750,16.8700,13.1200
2,2011-02,281,7.779359,3.0,20.385422,1.0,7.0,6.0,281,4.245409,...,1.63,4.95,3.32,281,15.121601,9.950,17.874355,4.130,17.7000,13.5700
3,2011-03,377,8.108753,3.0,15.529874,1.0,10.0,9.0,377,5.043050,...,1.25,4.95,3.70,377,19.584350,9.920,42.287673,3.480,17.7000,14.2200
4,2011-04,303,13.234323,4.0,40.515567,1.0,12.0,11.0,303,3.907657,...,1.06,4.13,3.07,303,19.721386,10.080,35.655175,3.765,17.7000,13.9350
5,2011-05,343,10.475219,3.0,42.157791,1.0,11.0,10.0,343,3.431341,...,1.25,4.13,2.88,343,15.656501,10.200,32.452504,4.130,17.3550,13.2250
6,2011-06,394,11.817259,3.5,29.259949,1.0,12.0,11.0,394,3.264213,...,1.25,4.13,2.88,394,21.260888,8.500,49.610423,3.405,17.8500,14.4450
7,2011-07,385,9.051948,3.0,18.433648,1.0,12.0,11.0,385,4.577247,...,0.85,3.29,2.44,385,15.758987,8.290,33.024404,2.980,18.7200,15.7400
8,2011-08,346,19.861272,4.0,110.729614,1.0,12.0,11.0,346,3.781416,...,1.25,4.13,2.88,346,26.912225,10.845,96.464237,4.130,19.8000,15.6700
9,2011-09,487,10.262834,4.0,23.119761,2.0,12.0,10.0,487,3.518255,...,1.25,4.13,2.88,487,20.046201,10.500,66.470839,4.130,19.8000,15.6700



✓ Grouped statistics saved to: ba_outputs/grouped_stats_by_InvoiceMonth.csv

FORMATTED GROUPED STATISTICS SUMMARY

GROUP: InvoiceMonth = 2010-12

  Quantity:
    Count:    412
    Mean:     6.0655
    Median:   2.0000
    Std Dev:  13.7546
    Q1:       1.0000
    Q3:       6.0000
    IQR:      5.0000

  UnitPrice:
    Count:    412
    Mean:     4.5847
    Median:   2.5500
    Std Dev:  16.0534
    Q1:       1.4500
    Q3:       4.2400
    IQR:      2.7900

  SalesAmount:
    Count:    412
    Mean:     14.0836
    Median:   7.5300
    Std Dev:  31.8882
    Q1:       3.3200
    Q3:       17.0000
    IQR:      13.6800

GROUP: InvoiceMonth = 2011-01

  Quantity:
    Count:    358
    Mean:     9.1704
    Median:   3.0000
    Std Dev:  19.1121
    Q1:       1.0000
    Q3:       8.0000
    IQR:      7.0000

  UnitPrice:
    Count:    358
    Mean:     4.8392
    Median:   2.2850
    Std Dev:  25.0216
    Q1:       1.2500
    Q3:       4.1300
    IQR:      2.8800

  SalesAmount:
    Count

,InvoiceMonth,Quantity_mean,UnitPrice_mean,SalesAmount_mean
0,2010-12,6.065534,4.584660,14.083592
1,2011-01,9.170391,4.839162,17.116313
2,2011-02,7.779359,4.245409,15.121601
3,2011-03,8.108753,5.043050,19.584350
4,2011-04,13.234323,3.907657,19.721386
5,2011-05,10.475219,3.431341,15.656501
6,2011-06,11.817259,3.264213,21.260888
7,2011-07,9.051948,4.577247,15.758987
8,2011-08,19.861272,3.781416,26.912225
9,2011-09,10.262834,3.518255,20.046201



QUICK COMPARISON: MEDIAN VALUES ACROSS GROUPS


,InvoiceMonth,Quantity_median,UnitPrice_median,SalesAmount_median
0,2010-12,2.0,2.550,7.530
1,2011-01,3.0,2.285,8.445
2,2011-02,3.0,2.550,9.950
3,2011-03,3.0,2.100,9.920
4,2011-04,4.0,2.100,10.080
5,2011-05,3.0,2.100,10.200
6,2011-06,3.5,2.100,8.500
7,2011-07,3.0,1.950,8.290
8,2011-08,4.0,2.080,10.845
9,2011-09,4.0,2.080,10.500


In [5]:
# ===============================
# PART 2 – VISUALIZATIONS
# step 1. Box-and-whisker plots (with mean marker)
# ===============================

import matplotlib.pyplot as plt
import os

# ---------- CREATE OUTPUT FOLDER ----------
os.makedirs("ba_outputs/boxplots", exist_ok=True)

print("\n" + "="*80)
print("CREATING BOX-AND-WHISKER PLOTS")
print("="*80)

# ---------- GENERATE BOX PLOTS ----------
for col in numeric_cols:
    print(f"\nGenerating box plot for: {col}")
    
    # Drop missing values
    series_clean = df_analysis[col].dropna()
    
    if len(series_clean) == 0:
        print(f"  ⚠ Warning: No valid data for {col}. Skipping.")
        continue
    
    # Compute mean for overlay
    mean_val = series_clean.mean()
    
    # Create figure
    fig, ax = plt.subplots(figsize=(6, 8))
    
    # Create vertical box plot
    bp = ax.boxplot(
        [series_clean],
        vert=True,
        patch_artist=True,
        tick_labels=[col],
        boxprops=dict(facecolor='lightblue', edgecolor='blue'),
        medianprops=dict(color='red', linewidth=2),
        whiskerprops=dict(color='blue'),
        capprops=dict(color='blue'),
        flierprops=dict(marker='o', markerfacecolor='gray', markersize=4, alpha=0.5)
    )
    
    # Overlay mean marker at x=1 (the position of the box)
    ax.plot(
        1, 
        mean_val, 
        marker='D',  # Diamond marker
        color='green', 
        markersize=10,
        markeredgecolor='darkgreen',
        markeredgewidth=1.5,
        label='Mean',
        zorder=5  # Ensure it appears on top
    )
    
    # Add legend
    ax.legend(loc='upper right', fontsize=10)
    
    # Labels and title
    ax.set_ylabel(col, fontsize=12)
    ax.set_title(f"Box-and-Whisker Plot: {col}", fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Save figure
    output_filename = f"{col}_box.png"
    output_path = os.path.join("ba_outputs", "boxplots", output_filename)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Saved: {output_path}")
    print(f"    Mean value: {mean_val:.4f}")
    print(f"    Sample size: {len(series_clean):,}")

print("\n" + "="*80)
print("✓ ALL BOX PLOTS COMPLETED")
print("="*80)
print(f"Output folder: ba_outputs/boxplots/")
print(f"Files created: {len(numeric_cols)} PNG files")


CREATING BOX-AND-WHISKER PLOTS

Generating box plot for: Quantity
  ✓ Saved: ba_outputs/boxplots/Quantity_box.png
    Mean value: 10.0606
    Sample size: 5,398

Generating box plot for: UnitPrice
  ✓ Saved: ba_outputs/boxplots/UnitPrice_box.png
    Mean value: 4.6845
    Sample size: 5,398

Generating box plot for: SalesAmount
  ✓ Saved: ba_outputs/boxplots/SalesAmount_box.png
    Mean value: 18.4337
    Sample size: 5,398

✓ ALL BOX PLOTS COMPLETED
Output folder: ba_outputs/boxplots/
Files created: 3 PNG files


In [6]:
# ===============================
# PART 2 – VISUALIZATIONS
# step 2. Histograms (with mean line)
# ===============================

import matplotlib.pyplot as plt
import os

# ---------- CREATE OUTPUT FOLDER ----------
os.makedirs("ba_outputs/histograms", exist_ok=True)

print("\n" + "="*80)
print("CREATING HISTOGRAMS")
print("="*80)

# ---------- GENERATE HISTOGRAMS ----------
for col in numeric_cols:
    print(f"\nGenerating histogram for: {col}")
    
    # Drop missing values
    series_clean = df_analysis[col].dropna()
    
    if len(series_clean) == 0:
        print(f"  ⚠ Warning: No valid data for {col}. Skipping.")
        continue
    
    # Compute mean for vertical line
    mean_val = series_clean.mean()
    
    # Create figure
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot histogram with default bins
    n, bins, patches = ax.hist(
        series_clean,
        bins='auto',  # default automatic binning
        color='skyblue',
        edgecolor='black',
        alpha=0.7,
        linewidth=0.5
    )
    
    # Add vertical line at mean
    ax.axvline(
        mean_val,
        color='red',
        linestyle='--',
        linewidth=2,
        label='Mean',
        zorder=5
    )
    
    # Add legend
    ax.legend(loc='upper right', fontsize=11)
    
    # Labels and title
    ax.set_xlabel(col, fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)
    ax.set_title(f"Histogram: {col}", fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Save figure
    output_filename = f"{col}_hist.png"
    output_path = os.path.join("ba_outputs", "histograms", output_filename)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Saved: {output_path}")
    print(f"    Mean value: {mean_val:.4f}")
    print(f"    Sample size: {len(series_clean):,}")
    print(f"    Number of bins: {len(bins)-1}")
    print(f"    Value range: [{series_clean.min():.4f}, {series_clean.max():.4f}]")

print("\n" + "="*80)
print("✓ ALL HISTOGRAMS COMPLETED")
print("="*80)
print(f"Output folder: ba_outputs/histograms/")
print(f"Files created: {len(numeric_cols)} PNG files")


CREATING HISTOGRAMS

Generating histogram for: Quantity
  ✓ Saved: ba_outputs/histograms/Quantity_hist.png
    Mean value: 10.0606
    Sample size: 5,398
    Number of bins: 1793
    Value range: [-144.0000, 1900.0000]

Generating histogram for: UnitPrice
  ✓ Saved: ba_outputs/histograms/UnitPrice_hist.png
    Mean value: 4.6845
    Sample size: 5,398
    Number of bins: 6930
    Value range: [0.0600, 2275.5400]

Generating histogram for: SalesAmount
  ✓ Saved: ba_outputs/histograms/SalesAmount_hist.png
    Mean value: 18.4337
    Sample size: 5,398
    Number of bins: 3351
    Value range: [-2185.0400, 3285.0000]

✓ ALL HISTOGRAMS COMPLETED
Output folder: ba_outputs/histograms/
Files created: 3 PNG files


In [7]:
# ===============================
# PART 2 – VISUALIZATIONS
# step 3. Zoomed plots (within fences)
# ===============================

import matplotlib.pyplot as plt
import os
import numpy as np

# ---------- CREATE OUTPUT FOLDERS ----------
os.makedirs("ba_outputs/boxplots_zoom", exist_ok=True)
os.makedirs("ba_outputs/histograms_zoom", exist_ok=True)

print("\n" + "="*80)
print("CREATING ZOOMED VISUALIZATIONS (WITHIN FENCES)")
print("="*80)

# ---------- HELPER FUNCTION: COMPUTE FENCES ----------
def compute_fences(series):
    """
    Compute lower and upper fences for outlier detection.
    Lower fence = Q1 - 1.5 * IQR
    Upper fence = Q3 + 1.5 * IQR
    """
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_fence = q1 - 1.5 * iqr
    upper_fence = q3 + 1.5 * iqr
    return lower_fence, upper_fence


# ---------- GENERATE ZOOMED PLOTS ----------
for col in numeric_cols:
    print(f"\n{'='*80}")
    print(f"Processing zoomed plots for: {col}")
    print('='*80)
    
    # Drop missing values
    series_clean = df_analysis[col].dropna()
    
    if len(series_clean) == 0:
        print(f"  ⚠ Warning: No valid data for {col}. Skipping.")
        continue
    
    # Compute fences
    lower_fence, upper_fence = compute_fences(series_clean)
    
    # Filter data within fences
    series_filtered = series_clean[(series_clean >= lower_fence) & (series_clean <= upper_fence)]
    
    if len(series_filtered) == 0:
        print(f"  ⚠ Warning: No data within fences for {col}. Skipping.")
        continue
    
    # Compute mean of filtered data
    mean_filtered = series_filtered.mean()
    
    print(f"  Original data: {len(series_clean):,} values")
    print(f"  Lower fence: {lower_fence:.4f}")
    print(f"  Upper fence: {upper_fence:.4f}")
    print(f"  Filtered data: {len(series_filtered):,} values ({100*len(series_filtered)/len(series_clean):.1f}%)")
    print(f"  Mean (filtered): {mean_filtered:.4f}")
    
    # ========================================
    # BOX PLOT (ZOOMED)
    # ========================================
    print(f"\n  Creating zoomed box plot...")
    
    fig, ax = plt.subplots(figsize=(6, 8))
    
    # Create vertical box plot
    bp = ax.boxplot(
        [series_filtered],
        vert=True,
        patch_artist=True,
        tick_labels=[f"{col}\n(within fences)"],
        boxprops=dict(facecolor='lightgreen', edgecolor='darkgreen'),
        medianprops=dict(color='red', linewidth=2),
        whiskerprops=dict(color='darkgreen'),
        capprops=dict(color='darkgreen'),
        flierprops=dict(marker='o', markerfacecolor='gray', markersize=4, alpha=0.5)
    )
    
    # Overlay mean marker at x=1
    ax.plot(
        1,
        mean_filtered,
        marker='D',  # Diamond marker
        color='blue',
        markersize=10,
        markeredgecolor='darkblue',
        markeredgewidth=1.5,
        label='Mean',
        zorder=5
    )
    
    # Add legend
    ax.legend(loc='upper right', fontsize=10)
    
    # Labels and title
    ax.set_ylabel(col, fontsize=12)
    ax.set_title(f"Box-and-Whisker (Zoomed): {col}", fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Save box plot
    output_filename = f"{col}_box_zoom.png"
    output_path = os.path.join("ba_outputs", "boxplots_zoom", output_filename)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Saved box plot: {output_path}")
    
    # ========================================
    # HISTOGRAM (ZOOMED)
    # ========================================
    print(f"  Creating zoomed histogram...")
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot histogram with default bins
    n, bins, patches = ax.hist(
        series_filtered,
        bins='auto',
        color='lightcoral',
        edgecolor='black',
        alpha=0.7,
        linewidth=0.5
    )
    
    # Add vertical line at mean
    ax.axvline(
        mean_filtered,
        color='red',
        linestyle='--',
        linewidth=2,
        label='Mean',
        zorder=5
    )
    
    # Add legend
    ax.legend(loc='upper right', fontsize=11)
    
    # Labels and title
    ax.set_xlabel(col, fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)
    ax.set_title(f"Histogram (Zoomed within fences): {col}", fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Save histogram
    output_filename = f"{col}_hist_zoom.png"
    output_path = os.path.join("ba_outputs", "histograms_zoom", output_filename)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Saved histogram: {output_path}")

print("\n" + "="*80)
print("✓ ALL ZOOMED VISUALIZATIONS COMPLETED")
print("="*80)
print(f"Box plots saved to: ba_outputs/boxplots_zoom/")
print(f"Histograms saved to: ba_outputs/histograms_zoom/")
print(f"Files created: {2 * len(numeric_cols)} PNG files")


CREATING ZOOMED VISUALIZATIONS (WITHIN FENCES)

Processing zoomed plots for: Quantity
  Original data: 5,398 values
  Lower fence: -14.0000
  Upper fence: 26.0000
  Filtered data: 5,107 values (94.6%)
  Mean (filtered): 5.7652

  Creating zoomed box plot...
  ✓ Saved box plot: ba_outputs/boxplots_zoom/Quantity_box_zoom.png
  Creating zoomed histogram...
  ✓ Saved histogram: ba_outputs/histograms_zoom/Quantity_hist_zoom.png

Processing zoomed plots for: UnitPrice
  Original data: 5,398 values
  Lower fence: -3.0700
  Upper fence: 8.4500
  Filtered data: 4,997 values (92.6%)
  Mean (filtered): 2.5287

  Creating zoomed box plot...
  ✓ Saved box plot: ba_outputs/boxplots_zoom/UnitPrice_box_zoom.png
  Creating zoomed histogram...
  ✓ Saved histogram: ba_outputs/histograms_zoom/UnitPrice_hist_zoom.png

Processing zoomed plots for: SalesAmount
  Original data: 5,398 values
  Lower fence: -18.1000
  Upper fence: 39.1800
  Filtered data: 4,921 values (91.2%)
  Mean (filtered): 10.2413

  Crea